In [1]:
import tensorflow as tf

In [2]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import xgboost as xgb
import itertools
import os

%matplotlib inline

In [7]:
import feather
path = 'AppTesting.Feather'
df = feather.read_dataframe(path)
df = df.loc[:]
X= df.iloc[:, 1:]
y= df.iloc[:,0]

,TRIAGE_CATEGORY,AGE,Male,Female,PATIENT_HEIGHT,PATIENT_WEIGHT,SMOKING_STATUS,PREGNANCY_STATUS,AVPU_1,GCS_1,RR_1,PULSE_1,HEART_RATE_1,O2SATS_1
0,3.0,50.0,0.0,1.0,160.739073,77.941150,0.0,0.0,1.000000,14.896529,18.000000,97.000000,81.791940,100.000000
1,3.0,49.0,1.0,0.0,174.663449,91.375446,0.0,0.0,1.039966,14.864701,18.791008,86.417111,81.673813,98.129391
2,2.0,55.0,1.0,0.0,172.688899,87.663757,0.0,0.0,1.036000,14.866306,24.000000,108.000000,80.791153,91.000000
3,3.0,62.0,1.0,0.0,171.289709,86.747946,0.0,0.0,1.000000,15.000000,20.000000,85.000000,81.075669,93.000000
4,3.0,50.0,1.0,0.0,172.688899,87.663757,0.0,0.0,1.000000,14.866306,18.000000,75.000000,80.791153,99.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556293,4.0,44.0,1.0,0.0,174.663449,91.375446,0.0,0.0,1.000000,15.000000,18.000000,67.000000,81.673813,97.000000
556294,3.0,63.0,1.0,0.0,171.289709,86.747946,0.0,0.0,1.034461,15.000000,18.000000,72.000000,81.075669,97.000000
556295,5.0,50.0,1.0,0.0,172.688899,87.663757,0.0,0.0,1.036000,15.000000,18.000000,79.000000,80.791153,97.000000
556296,2.0,84.0,0.0,1.0,155.000000,75.000000,0.0,1.0,1.000000,15.000000,16.000000,67.000000,67.000000,100.000000


In [16]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


In [18]:
#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True, 
                                                    stratify = y, 
                                                    random_state = 3)


In [19]:
X_train.shape
#X_train[:-1]

(389408, 13)

In [21]:
ncol = len(X_train.columns)
# input shape

In [22]:
train_class = y_train
test_class = y_test
encoder = LabelEncoder()
encoder.fit(train_class)
y_train = encoder.transform(train_class)
y_test = encoder.transform(test_class)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [24]:
batch_size = 32
epochs = 2

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(ncol,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 350467 samples, validate on 38941 samples
Epoch 1/2
350467/350467 [==============================] - 42s 120us/step - loss: 1.4028 - accuracy: 0.3819 - val_loss: 1.2606 - val_accuracy: 0.3983
Epoch 2/2
350467/350467 [==============================] - 42s 119us/step - loss: 1.2806 - accuracy: 0.3899 - val_loss: 1.2585 - val_accuracy: 0.4093


In [26]:
score = model.evaluate(X_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

166890/166890 [==============================] - 6s 34us/step
Test accuracy: 0.41001856327056885


In [27]:
model.save('KERAS_TEST')